<a href="https://colab.research.google.com/github/RayMelgares/Math-152/blob/main/Reciprocity_Euler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction: Exploration of Reciprocity Laws in the Spirit of Euler.

In this exploration, we will be examining which primes can be written as a sum of squares in the form: $p=x^2+Ny^2$, where $p$ is an odd prime and $N$ is a positive integer. This is an extention from Fermat's "Christmas Theorem" which states that an odd prime $p$ can be expressed as a sum of two squares if and only if $p \,\%\, 4$ is congruent to 1. Euler later looked at $p=x^2+Ny^2$ and like Fermat, found what we call a reciprocity law, a theorem saying that a prime can be written as $p=x^2+Ny^2$ if and only if $p \,\%\ M$ belongs to a finite set. This exploration will be using code to follow in his footsteps, focusing on $M=4N$. The following code below gives us a list of primes of up to a given value, using a method similair to the Sieve of Eratosthenes. We will choose the primes up to $1000000$, the first $78498$ primes.






In [5]:
from math import sqrt

def isprime_list(n):
    ''' 
    Return a list of length n+1
    with Trues at prime indices and Falses at composite indices.
    '''
    flags = [True] * (n+1)  # A list [True, True, True,...] to start.
    flags[0] = False  # Zero is not prime.  So its flag is set to False.
    flags[1] = False  # One is not prime.  So its flag is set to False.
    flags[4::2] = [False] * ((n-2)//2)
    p = 3
    while p <= sqrt(n):  # We only need to sieve by p is p <= sqrt(n).
        if flags[p]:  # We sieve the multiples of p if flags[p]=True.
            flags[p*p::2*p] = [False] * ((n-p*p)//(2*p)+1) # Sieves out multiples of p, starting at p*p.
        p = p + 2 # Try the next value of p.
        
    return flags

def where(L):
    '''
    Take a list of booleans as input and
    outputs the list of indices where True occurs.
    '''
    return [n for n in range(len(L)) if L[n]]

primes = where(isprime_list(1000000)) # The primes up to 1 million, in a list.

###1. Function for Primes That Can Be Expressed as $x^2 + Ny^2$

To start off, while looking at the `Euler` function we can examine $p=x^2+Ny^2$ by using a brute force approach by iterating through all possible $x$ and $y$ values. However, with the `Euler_alt` function, instead of using two loops we can use a single loop on the y variable to deduce the number of values needing to be tested. By adding the sqrt function we can cut the number of operations needing to be ran and we can see through the `%timeit` how effective this becomes.

In [6]:
from math import sqrt,floor,ceil

def Euler(p, N): # O(n) pretty sure ~sqrt(n) * sqrt(n)
  x_max = floor(sqrt(p))
  y_max = ceil(sqrt(p) / N)
  x = 1
  y = 1
  while(x <= x_max):
    while(y <= y_max):
      if(x**2 + N*y**2 == p):
        return True
      y += 1
    y = 1
    x += 1

  return False

def Euler_alt(p, N): # faster, O(n^(1/2))
  y = sqrt(p // N) // 1 # set y to be an integer so that Ny^2 <= p
  while(y > 0):
    sqrt_of_remainder = sqrt(p - N*y**2) #find remainder of p - Ny^2 and sqrt it to find x
    if(sqrt_of_remainder % 1 == 0 and sqrt_of_remainder > 0): # check that x is int and not 0
      return True
    y -= 1

  return False



%timeit Euler(10001,7)
%timeit Euler_alt(10001, 7)

882 µs ± 9.95 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
17 µs ± 3.72 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


###2. Primes of the form $p=x^2 +Ny^2$ and the remainders $p \, \%\ 4N$
 When checking for multiple different $N$'s of positive integers, (2, 3, 4, 5, 6, 7 and more) it can be seen that not all possible remainders of $p \,\%\, 4N$  occur. First, we observed that all remainders are odd, and that all $N$'s appear to have $1$ as a possible remainder. Furthermore, the upper bound for the remainders is naturally $4N-1$ as going any higher wouldn't be possible. As for the number of remainders, we observed that as $N$ grows in size the possibilities of the number of remainders also grows, and hence in some cases there are more possible remainders with a larger $N$. However, this correlation is rather weak and you cannot necessarlily say that for a larger $N$ there are a larger amount of remainders. For example, when we try $N=9$ we get $1,13,25$ as our remainders. Looking at $N=80$ we get $16$ different remainders, a fair amount more than  for $N=9$.  But if we try $N=90$ we get a smaller amount of only 12 different remainders.

###3. New Reciprocity Laws

To create a counter for the number of remainders occuring we have a function that inputs our number $N$, and how many primes we want to check. So if we input `check_primes_with_Euler(3, 1000)` we are setting our $N$ to $3$ and looking at the first $1000$ primes. We start with a list that we call primes that succeed, and a dictionary to count the remainders. We check of our primes and if our `Euler_alt` function outputs true we add that `p` to the list. For each of our succeeding primes, we check their remainder and note which remainder we get. At the end we print the remainders and the number of times they occured.

In [7]:
def check_primes_with_Euler(N, number_of_primes_to_check):
  primes_that_succeed = []
  remainders = {} #could also be implemented with an array where we add onto an index
  for p in primes[:number_of_primes_to_check]: #check a certain number of primes
    if (Euler_alt(p, N)): # if primes satisfies condition add to list
      primes_that_succeed.append(p)
      
  for p in primes_that_succeed: # for satisfactory primes, check remainder
    r = p % (4 * N)
    try:  #if remainder exists in dictionary, add to its count, otherwise set to 1
      remainders[r] += 1
    except:
      remainders[r] = 1

  print(remainders)
check_primes_with_Euler(3, 1000)

{7: 249, 1: 241}




Observation: The number of times each remainder appears is very consistent. For example, when checking the first $1000$ primes with $N = 3$, there is a remainder of $1$, $241$ times and a remainder of $7$, $249$ times.

####It appears as if the following is true.

When $N = 2$, $p$ can be written as $x^2 + 2y^2$ if and only if $p\,\%\,8$ is congruent to 1 or 3.

When $N = 3$, $p$ can be written as $x^2 + 3y^2$ if and only if $p\,\%\,12$ is congruent to 1 or 7.

When $N = 4$, $p$ can be written as $x^2 + 4y^2$ if and only if $p\,\%\,16$ is congruent to 1, 5, 9 or 13.

When $N = 5$, $p$ can be written as $x^2 + 5y^2$ if and only if $p\,\%\,20$ is congruent to 1 or 9.

When $N = 6$, $p$ can be written as $x^2 + 6y^2$ if and only if $p\,\%\,24$ is congruent to 1 or 7.

When $N = 7$, $p$ can be written as $x^2 + 7y^2$ if and only if $p\,\%\,28$ is congruent to 1, 9, 11, 15, 23 or 25.

When $N = 8$, $p$ can be written as $x^2 + 8y^2$ if and only if $p\,\%\,32$ is congruent to 1, 9, 17 or 25.

When $N = 9$, $p$ can be written as $x^2 + 9y^2$ if and only if $p\,\%\,36$ is congruent to 1, 13 or 25.

###4. Variants
In this code we examine from the formula $p = ax^2 + by^2$ that from the $a$ and $b$ variables we want to determine which one classifies as the bigger and which one is the smaller. From there we can use the big number to reduce the number of iterations to then use the sqrt function to identify the remainders. 

In [8]:
def Euler_a_b(p, a, b):
  big, small = (a, b) if a >= b else (b, a) # set big to reduce iterations
  y = sqrt(p // big) // 1
  while(y > 0):
    sqrt_of_remainder = sqrt((p - big*y**2) / small)
    if(sqrt_of_remainder % 1 == 0 and sqrt_of_remainder > 0):
      return True
    y -= 1

  return False

Euler_a_b(11, 3, 3)


False

In [9]:
def check_primes_with_Euler_a_b(a, b, number_of_primes_to_check):
  primes_that_succeed = []
  remainders = {}
  for p in primes[:number_of_primes_to_check]:
    if (Euler_a_b(p, a, b)):
      primes_that_succeed.append(p)
      
  for p in primes_that_succeed:
    r = p % (4 * a * b)
    try:
      remainders[r] += 1
    except:
      remainders[r] = 1

  print(remainders)
check_primes_with_Euler_a_b(4, 5, 78000)

{41: 1223, 61: 1224, 29: 1222, 69: 1221, 1: 1229, 9: 1203, 21: 1204, 49: 1202}


Observation: $a$ and $b$ cannot be multiples of one another. We can see this when considering that our function checks if $p = ax^2 + by^2$. WLOG, suppose $a = nb$ for some $n \in Z_{>0}$. Then: 

$$p = ax^2 + by^2$$
$$= bnx^2+by^2$$
$$= b(nx^2+y^2)$$

Therefore, $\frac{p}{b}$ would equal the integer $nx^2+y^2$ which of course implies that $p$ is divisible by $b$ and therefore not a prime.

####It seems that the following is true.

When $a = 2$ and $b = 3$, $p$ can be written as $2x^2 + 3y^2$ if and only if $p\,\%\,24$ is congruent to 5 or 11.

When $a = 2$ and $b = 5$, $p$ can be written as $2x^2 + 5y^2$ if and only if $p\,\%\,40$ is congruent to 7, 13, 23 or 37.

When $a = 3$ and $b = 5$, $p$ can be written as $3x^2 + 5y^2$ if and only if $p\,\%\,60$ is congruent to 17, 23, 47 or 53.

When $a = 4$ and $b = 5$, $p$ can be written as $4x^2 + 5y^2$ if and only if $p\,\%\,80$ is congruent to 1, 9, 21, 29, 41, 49, 61 or 69.

#### A quick exploration into the primes written as a sum of two cubes in the form: $p=x^3+Ny^3$.
 We can adjust our existing code for the sum of squares as a sum of cubes.

In [21]:
from numpy import cbrt 
def Euler_alt_cube(p, N): # Same as Euler_alt but with cubes
  y = cbrt(p // N) // 1 # set y to be an integer so that Ny^3 <= p
  while(y > 0):
    cbrt_of_remainder = cbrt(p - N*y**3) #find remainder of p - Ny^3 and cbrt it to find x
    if(cbrt_of_remainder % 1 == 0 and cbrt_of_remainder > 0): # check that x is int and not 0
      return True
    y -= 1

  return False
Euler_alt_cube(223,3)

False

In [20]:
def check_primes_with_Euler_cube(N, number_of_primes_to_check): 
  primes_that_succeed = []
  remainders = {} #could also be implemented with an array where we add onto an index
  for p in primes[:number_of_primes_to_check]: #check a certain number of primes
    if (Euler_alt_cube(p, N)): # if primes satisfies condition add to list
      primes_that_succeed.append(p)
      
  for p in primes_that_succeed: # for satisfactory primes, check remainder
    r = p % (4 * N)
    try:  #if remainder exists in dictionary, add to its count, otherwise set to 1
      remainders[r] += 1
    except:
      remainders[r] = 1

  print(remainders)
check_primes_with_Euler_cube(5,70000)

{13: 31, 1: 42, 19: 38, 3: 34, 7: 31, 17: 38, 11: 27, 9: 15}


Observations: 

Right away after experimenting with `Euler_alt_cube` we notice that the primes that can be expressed as $x^3+Ny^3$ are much less common than the squared case. This makes sense as when we input different  positive integers they will be further apart since $x^3$ grows faster. Also the pattern that the count of each remainder is roughly the same breaks down, with some remainders being more common than others.
#### With a bit of expermentation, it appears the following are true:





When $N=3$, if $p$ can be written as $x^3+3y^3$ then $p \,\%\, 12$ is congruent to $1,5,7,11$.

When $N=4$, if $p$ can be written as $x^3+4y^3$ then $p \,\%\, 16$ is congruent to $1,3,5,7,9,11,13,15$.

When $N=5$, if $p$ can be written as $x^3+5y^3$ then $p \,\%\ 20$ is congruent to $1,3,7,9,11,13,17,19$.


We can see that for the remainders of $4N$ these laws are not if and only if statements. The reciprocity laws for $4N$ lose their biconditionality. For example if$(N=3)$, $223 \%\ 12$ is congruent to $7$, but $223$ cannot be written in the form $x^3+3y^3$ according to our code. Also notice that all possible odd remainders occur when $N=4$ so the reciprocity law is quite redundant.

###5. Conclusion
Referring back to the modern perspective of Fermat’s “Christmas Theorem”, we were able to generate different codes that represent the reciprocity law. While examining the theorem of reciprocity law, which is, a prime number $p$ can be expressed as $x^2 + Ny^2$ if and only if $p \, \%\ M$ belongs to a certain finite set, we were able to show different codes to generate more reciprocity laws. As we look back onto these new codes that were created, we were able to examine the prime numbers and the remainders within the formula $x^2 +Ny^2$. Within these codes we then found how different methods, such as using the sqrt, can make a code more efficient with what it is we are wanting to find, which is what we can take comprehend from the above construct codes.   
